## 1. Import thư viện và nạp dữ liệu

In [ ]:
import sys
import os
import numpy as np

# Thêm thư mục src vào sys.path để import các module tự định nghĩa
sys.path.append(os.path.abspath(os.path.join('..')))

from src import data_processing as dp

## 2. Quy trình tiền xử lý

**Giai đoạn 1: Làm Sạch Dữ Liệu**
1.  **Loại bỏ các cột không cần thiết**: Các cột `enrollee_id` và `city` sẽ bị xóa vì không mang lại giá trị dự đoán.
2.  **Xử lý dữ liệu trùng lặp**: Các dòng bị trùng lặp hoàn toàn sẽ bị loại bỏ khỏi tập huấn luyện để tránh sai lệch cho mô hình. Bước này được bỏ qua đối với tập test.
3.  **Chuẩn hóa giá trị thiếu**: Các giá trị rỗng (`''`) trong tất cả các cột kiểu chuỗi sẽ được thay thế bằng một danh mục nhất quán là `Missing`.

**Giai đoạn 2: Feature Engineering và Mã hóa**

Quá trình này được áp dụng cho tất cả các cột phân loại, bao gồm `experience`, `education_level`, và các cột khác.

1.  **Tạo biến chỉ báo (Indicator Variables)**: Đối với mỗi cột có khả năng chứa giá trị `Missing`, một cột nhị phân mới (ví dụ: `education_level_is_missing`) được tạo ra. Cột này nhận giá trị `1` nếu giá trị ban đầu bị thiếu, và `0` trong trường hợp ngược lại.
2.  **Xử lý giá trị thiếu và Mã hóa Ordinal**:
    *   Các giá trị trong mỗi cột phân loại được ánh xạ tới các số nguyên có thứ tự.
    *   Đối với các giá trị `Missing`, chúng được thay thế bằng giá trị *median* của các giá trị đã được mã hóa trong cùng cột.
    *   Toàn bộ quá trình này được thực hiện trong một bước bằng cách cập nhật ánh xạ (mapping) cho giá trị `Missing` rồi áp dụng nó lên toàn bộ cột.

In [ ]:
import numpy as np
from numpy.lib import recfunctions as rfn

def preprocess_pipeline(data, is_test_set=False):
    """Một pipeline thực hiện tất cả các bước tiền xử lý dữ liệu."""
    
    # --- 1. Làm Sạch Dữ Liệu --- 
    data_cleaned = dp.remove_columns(data, ['enrollee_id', 'city'])
    
    if not is_test_set:
        data_cleaned = dp.remove_duplicates(data_cleaned)
    
    # Điền giá trị thiếu cho các cột categorical
    categorical_cols = [name for name, dtype in data_cleaned.dtype.fields.items() if dtype[0].kind in ('U', 'O')]
    data_cleaned = dp.fill_missing_categorical(data_cleaned, columns=categorical_cols, fill_value='Missing')

    # --- 2. Feature Engineering & Encoding --- 
    data_encoded = data_cleaned.copy() if hasattr(data_cleaned, 'copy') else data_cleaned[:]

    # Định nghĩa các mapping ordinal
    mappings = {
        'experience': {
            'Missing': -1, '<1': 0, '>20': 21,
            **{str(i): i for i in range(1, 21)}
        },
        'relevent_experience': {'No relevent experience': 0, 'Has relevent experience': 1},
        'education_level': {'Missing': -1, 'Primary School': 1, 'High School': 2, 'Graduate': 3, 'Masters': 4, 'Phd': 5},
        'last_new_job': {'Missing': -1, 'never': 0, '1': 1, '2': 2, '3': 3, '4': 4, '>4': 5},
        'company_size': {'Missing': -1, '<10': 0, '10/49': 1, '50-99': 2, '100-500': 3, '500-999': 4, '1000-4999': 5, '5000-9999': 6, '10000+': 7},
        'gender': {'Missing': -1, 'Female': 0, 'Male': 1, 'Other': 2},
        'enrolled_university': {'Missing': -1, 'no_enrollment': 0, 'Part time course': 1, 'Full time course': 2},
        'major_discipline': {'Missing': -1, 'No Major': 0, 'Humanities': 1, 'Arts': 2, 'Business Degree': 3, 'STEM': 4, 'Other': 5},
        'company_type': {'Missing': -1, 'NGO': 0, 'Public Sector': 1, 'Early Stage Startup': 2, 'Funded Startup': 3, 'Pvt Ltd': 4, 'Other': 5}
    }

    for col_name, mapping in mappings.items():
        if col_name not in data_encoded.dtype.names:
            continue
            
        # Tạo indicator cho giá trị Missing (nếu có)
        if 'Missing' in mapping:
            indicator_col = f"{col_name}_is_missing"
            is_missing = np.zeros(len(data_encoded), dtype=np.int8)
            missing_mask = data_encoded[col_name] == 'Missing'
            is_missing[missing_mask] = 1
            
            data_encoded = rfn.append_fields(
                data_encoded, indicator_col, is_missing, dtypes=np.int8, usemask=False
            )

            # Thay thế 'Missing' bằng median của các giá trị hợp lệ
            valid_values = [v for k, v in mapping.items() if k != 'Missing']
            if valid_values:
                median_val = int(np.median(valid_values))
                mapping['Missing'] = median_val
        
        # Áp dụng ordinal encoding
        data_encoded = dp.encode_ordinal(data_encoded, col_name, mapping)
    
    return data_encoded

## 3. Áp dụng Pipeline và xuất dữ liệu

Bây giờ chúng ta sẽ áp dụng pipeline trên cho cả hai tệp `aug_train.csv` và `aug_test.csv`, sau đó lưu kết quả đã xử lý vào thư mục `data/processed`.

In [ ]:
def process_and_save(input_filename, output_filename, is_test_set=False):
    """Tải, xử lý và lưu một tệp dữ liệu."""
    # Tải dữ liệu
    raw_filepath = f'../data/raw/{input_filename}'
    raw_data = dp.load_data(raw_filepath)
    print(f"Đã tải {input_filename} với shape: {raw_data.shape}")
    
    # Áp dụng pipeline tiền xử lý
    processed_data = preprocess_pipeline(raw_data, is_test_set=is_test_set)
    print(f"Tiền xử lý hoàn tất cho {input_filename}. Shape mới: {processed_data.shape}")
    
    # Lưu dữ liệu đã xử lý
    output_dir = '../data/processed'
    os.makedirs(output_dir, exist_ok=True)
    output_filepath = os.path.join(output_dir, output_filename)
    
    header = ','.join(processed_data.dtype.names)
    # Sử dụng '%g' cho float để loại bỏ các số 0 không cần thiết ở cuối
    fmt_mapping = {'f8': '%g', 'i4': '%d', 'i8': '%d'}
    fmt_string = [fmt_mapping.get(processed_data.dtype[name].str[1:], '%s') for name in processed_data.dtype.names]

    np.savetxt(output_filepath, processed_data, delimiter=',', fmt=fmt_string, header=header, comments='')
    print(f"Đã lưu dữ liệu đã xử lý vào: {output_filepath}")
    print("---")

In [ ]:
# Xử lý tập train
process_and_save('aug_train.csv', 'aug_train_processed.csv', is_test_set=False)

# Xử lý tập test
process_and_save('aug_test.csv', 'aug_test_processed.csv', is_test_set=True)